<a href="https://colab.research.google.com/github/mohamadmansourX/TreeBased_Carla/blob/main/June062022_Carla_Wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LeafNode ---> merge_conditions

if both has '<=' we sould choose the min value but its not working unless we choose max??? same to '>'

In [2]:
!git clone https://github.com/carla-recourse/CARLA.git
%cd CARLA

Cloning into 'CARLA'...
remote: Enumerating objects: 2210, done.
remote: Counting objects: 100% (1012/1012), done.
remote: Compressing objects: 100% (625/625), done.
remote: Total 2210 (delta 645), reused 482 (delta 387), pack-reused 1198
Receiving objects: 100% (2210/2210), 2.10 MiB | 16.13 MiB/s, done.
Resolving deltas: 100% (1241/1241), done.
/content/CARLA


In [3]:
!git reset --hard 87983228739b616f46481960a1da77e13b05c974
!git pull

HEAD is now at 8798322 Chore/fix ordering bug (#150)
Updating 8798322..648b2aa
Fast-forward
 README.md                                          |   4 +
 carla/evaluation/distances.py                      |   6 +-
 carla/models/catalog/ANN_TORCH/model_ann.py        |  54 ----
 carla/models/catalog/Linear_TORCH/model_linear.py  |  40 ---
 carla/plotting/__init__.py                         |   3 +
 carla/plotting/barplot.py                          |  27 ++
 carla/plotting/plotting.py                         | 102 ++++++++
 carla/plotting/stripplot.py                        |  62 +++++
 carla/plotting/swarmplot.py                        |  51 ++++
 .../catalog/actionable_recourse/model.py           |   3 +-
 carla/recourse_methods/catalog/cchvae/model.py     |   2 +-
 carla/recourse_methods/catalog/cem/model.py        |   3 +-
 carla/recourse_methods/catalog/clue/model.py       |   2 +-
 carla/recourse_methods/catalog/crud/model.py       |   4 +-
 carla/recourse_methods/catalog/dice/model

In [4]:
!git checkout 87983228739b616f46481960a1da77e13b05c974

Note: checking out '87983228739b616f46481960a1da77e13b05c974'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 8798322 Chore/fix ordering bug (#150)


In [5]:
%%bash
echo """pip install -U pip setuptools wheel
pip install -e .
pip install -r requirements-dev.txt
pre-commit install
pip install -r requirements-dev.txt
""" > mm_bash_setup.sh

In [6]:
!bash mm_bash_setup.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 46.7 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/CARLA
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
!pip uninstall pandas -y
!pip install pandas==1.3.5

Found existing installation: pandas 1.1.4
Uninstalling pandas-1.1.4:
  Successfully uninstalled pandas-1.1.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 53.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.4 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.22.0 which is incompatible.
fastai 2.6.3 requires torchvision>=0.8.2, but you have torchvision 0.8.1 which is incompatible.


# Initializing the Model

In [8]:
%cd /content/CARLA

/content/CARLA


In [9]:
from typing import Dict, List, Tuple, Union
import pandas as pd
from carla import RecourseMethod
from carla.data.api import data, Data
from carla.models.api import MLModel
from carla.recourse_methods.autoencoder import (
    VAEDataset,
    VariationalAutoencoder,
    train_variational_autoencoder,
)
from carla.recourse_methods.processing import (
    check_counterfactuals,
    merge_default_parameters,
    reconstruct_encoding_constraints,
)
# For Descision Tree implementation
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from carla import Benchmark
from carla.recourse_methods import Dice, Face
import warnings
warnings.simplefilter("ignore", UserWarning)
import numpy as np
import random
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [10]:
import yaml
def load_setup() -> Dict:
    with open("experimental_setup.yaml", "r") as f:
        setup_catalog = yaml.safe_load(f)
    return setup_catalog["recourse_methods"]
setup = load_setup()

In [11]:
"""
Tree Leaf utils
"""

# !git clone https://github.com/carla-recourse/CARLA.git


import enum
from typing import Dict, List, Tuple, Union
import pandas as pd
from carla import RecourseMethod
from carla.data.api import data, Data
from carla.models.api import MLModel
from carla.recourse_methods.autoencoder import (
    VAEDataset,
    VariationalAutoencoder,
    train_variational_autoencoder,
)
from carla.recourse_methods.processing import (
    check_counterfactuals,
    merge_default_parameters,
    reconstruct_encoding_constraints,
)
# For Descision Tree implementation
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from carla import Benchmark
from carla.recourse_methods import Dice, Face
import warnings
warnings.simplefilter("ignore", UserWarning)
import numpy as np
import random
from sklearn.model_selection import train_test_split
from copy import deepcopy
class LeafNode:
    def __init__(self, conditions, label, weight):
        # Conditions is a list of tuples from the root node to the leaf node
        self.conditions = deepcopy(conditions)
        # Label is the label of the leaf node
        self.label = label
        # Wieght Either entropy or gini
        self.weight = weight
        # Duplicate conditions
        self.duplicate_conditions = []

    def __repr__(self):
        """
        Print the leaf node with conditions and label
        """
        return "LeafNode(label={}, weight={}, conditions={})".format(self.label, self.weight, self.conditions)

    def compare_node(self, other): #TODO misleading name
        """
        Get the distance between two leaf nodes by returning a set of conditions as follows:
        1. Initialize conditions as other conditions
        2. Remove conditions that are exactly the same with self
        3. Return the remaining conditions

        # TODO common feature
        """
        # Initialize conditions as other conditions
        conditions = other.conditions
        # Remove conditions that are common with self
        for condition in self.conditions:
            conditions = [c for c in conditions if c != condition]
        # Return the remaining conditions
        return conditions

    def merge_conditions(self):
        """
        If there are two conditions with the same feature and threshold_sign, merge them into one condition
        """
        # Initialize conditions as other conditions
        # 
        conditions = self.conditions
        # indexes to be dropped
        indexes = []
        # Search for conditions with the same feature and threshold_sign
        for i in range(len(conditions)):
            # if i in the indexes to be dropped, skip
            if i in indexes:
                continue
            for j in range(i + 1, len(conditions)):
                # if j in the indexes to be dropped, skip
                if j in indexes:
                    continue
                if conditions[i].feature == conditions[j].feature:
                    if conditions[i].threshold_sign == conditions[j].threshold_sign:
                        # Merge the two conditions
                        if conditions[i].threshold_sign == '<=':
                            conditions[i].threshold = min(conditions[i].threshold, conditions[j].threshold)
                        else:
                            conditions[i].threshold = max(conditions[i].threshold, conditions[j].threshold)
                        # Add index to drop
                        indexes.append(j)
                    else:
                        # Add it to duplicate conditions
                        if conditions[j].feature not in self.duplicate_conditions:
                            self.duplicate_conditions.append(conditions[j].feature)
        # Drop indexes from conditions
        conditions = [c for i, c in enumerate(conditions) if i not in indexes]
        self.conditions = conditions
        self.duplicate_conditions = list(set(self.duplicate_conditions))

    def check_point(self, point):
        """
        Check if the point satisfies the conditions of the leaf node
        """
        # Check if the point satisfies the conditions of the leaf node
        for condition in self.conditions:
            if not condition.check_point(point):
                return False
        return True

    def generate_point(self, point, data_catalog = None, sigma =0.5, gamma = 0):
        """
        Generate a point from a point
        """
        # loop through the duplicate conditions
        for feature in self.duplicate_conditions:
            # get the two conditions with that feature
            conditions = [c for c in self.conditions if c.feature == feature]
            # data_catalog contains {'categorical':[],'continuous':[],'imutable':[], 'continuous_stats':[]}
            if feature in data_catalog['categorical']:
                # Assert that there shouldn't be duplicate conditions for a binary feature (categorical here are binaries)
                assert False, "There shouldn't be duplicate conditions for a binary feature"
                #TODO (general user he can't solve it)
                # Thrsh can be continous, then generate a random point between threshold and round the result
            elif feature in data_catalog['continuous']:
                # using the continuous_stats get the std and mean
                std = data_catalog['continuous_stats'][feature]['std']
                mean = data_catalog['continuous_stats'][feature]['mean']
                minn = data_catalog['continuous_stats'][feature]['min']
                maxx = data_catalog['continuous_stats'][feature]['max']
                # Using the mean, std, min and max create a bias value
                # bias values is std/10 * (max - min)
                bias = std / sigma # TOCHECK LATER
                bias = min(bias, abs(conditions[0].threshold - conditions[1].threshold))
                # Min
                if gamma == 0:
                    min_bias = 0
                else:
                    min_bias = std / gamma
                # Generate a random value between the two thresholds
                bias = random.uniform(min_bias, bias)
                # Add the bias to the threshold
                if conditions[0].threshold_sign == '<=':
                    point[feature] = conditions[0].threshold + bias
                else:
                    point[feature] = conditions[1].threshold + bias
        for condition in self.conditions:
            if condition.feature not in self.duplicate_conditions:
                if condition.feature in data_catalog['categorical']:
                    # Simply flip the value
                    # Round the threshold
                    point[condition.feature] = not point[condition.feature]
                else: # condition.feature in data_catalog['continuous']:
                    std = data_catalog['continuous_stats'][condition.feature]['std']
                    mean = data_catalog['continuous_stats'][condition.feature]['mean']
                    minn = data_catalog['continuous_stats'][condition.feature]['min']
                    maxx = data_catalog['continuous_stats'][condition.feature]['max']
                    bias = std / sigma
                    # Min
                    if gamma == 0:
                        min_bias = 0
                    else:
                        min_bias = std / gamma
                    # Generate a random value between the two thresholds
                    bias = random.uniform(min_bias, bias)
                    if condition.threshold_sign == '<=':
                        point[condition.feature] = condition.threshold + bias
                    else:
                        point[condition.feature] = condition.threshold - bias
        return point


class Condition:
    def __init__(self, feature, threshold, threshold_sign):
        # Feature is the feature name
        self.feature = feature
        # Value is the value of the feature
        self.threshold = threshold
        # <= or > since they are the only two threshold_sign in Decision Tree
        self.threshold_sign = threshold_sign
    def __repr__(self):
        return f'{self.feature} {self.threshold_sign} {self.threshold}'
    def check_point(self, point):
        """
        Check if the point satisfies the condition
        """
        # Check if the point satisfies the condition
        if self.threshold_sign == '<=':
            return point[self.feature] <= self.threshold
        else:
            return point[self.feature] > self.threshold


class TreeLeafs:
    def __init__(self, tree, feature_input_order):
        self.tree = tree
        self.feature_input_order = feature_input_order
        self.leafs_nodes = []
        self.get_leaf_nodes(tree)
        for leaf in self.leafs_nodes:
            leaf.merge_conditions()

    def get_leaf_nodes(self, tree, node_id=0, conditions=[]):
        """
        This will be a recursion function that will append to leaf_nodes list, their labels and set of conditions
        If the node is a leaf node, then it will append a LeafNode object to leaf_nodes
        If the node is not a leaf node, then it will return while adding the conditions of the left and right child to the list
        """
        # If the node is a leaf node
        if tree.children_left[node_id] == -1 and tree.children_right[node_id] == -1:
            # Append the leaf node to the list
            self.leafs_nodes.append(LeafNode(conditions, np.argmax(tree.value[node_id]), tree.impurity[node_id]))
        # If the node is not a leaf node
        else:
            # Need to get the feature of the node
            feature = self.feature_input_order[tree.feature[node_id]]
            # Need to get the threshold of the node
            threshold = tree.threshold[node_id]
            # For right child if exists, threshold_sign is >
            if tree.children_right[node_id] != -1:
                conditions_right = conditions.copy()
                # Append the condition to the list
                conditions_right.append(Condition(feature, threshold, '>'))
                # Get the right child
                self.get_leaf_nodes(tree, tree.children_right[node_id], conditions_right)
            # For left child if exists, threshold_sign is <=
            if tree.children_left[node_id] != -1:
                conditions_left = conditions.copy()
                # Append the condition to the list
                conditions_left.append(Condition(feature, threshold, '<='))
                # Get the left child
                self.get_leaf_nodes(tree, tree.children_left[node_id], conditions_left)


In [12]:
from tqdm.notebook import tqdm
tqdm.pandas()


class TreeBasedContrastiveExplanation(RecourseMethod):
    '''
    Decision Tree Based contrastive explanations
    '''
    _DEFAULT_HYPERPARAMS = {
      "data_name": None,
      "n_search_samples": 300,
      "p_norm": 1,
      "step": 0.1,
      "max_iter": 1000,
      "clamp": True,
      "target_class": [0, 1],
      "binary_cat_features": True,
      "vae_params": {
          "layers": None,
          "train": True,
          "lambda_reg": 1e-6,
          "epochs": 5,
          "lr": 1e-3,
          "batch_size": 32,
      },
      "tree_params": {
          "min_entries_per_label": 1000,
          "grid_search_jobs": -1,
          "min_weight_gini": 100, # set to 0.5 since here both class have same prob
          "max_search": 500,
          "grid_search": {
                "splitter": ["best"],
                "criterion": ["gini"],
                "max_depth": [6],
                "min_samples_split": [2],
                "min_samples_leaf": [1],
                "max_features": [None] #Note changing this will result in removing features that we might want to keep
          }
      }

    }

    def __init__(self, dataset:Data, mlmodel: MLModel, hyperparams: Dict, data_catalog: Dict):
        super().__init__(mlmodel)
        # Construct catalog
        self.data_catalog = data_catalog
        # Construct mlmodel
        self.mlmodel = mlmodel
        # Construct the hyperparameters
        self.hyperparams = merge_default_parameters(hyperparams, self._DEFAULT_HYPERPARAMS)
        # Construct the VAE
        # self.vae = TEMP_VAE
        self.vae = self.load_vae(dataset, self.hyperparams["vae_params"], mlmodel, mlmodel.data.name)
        # Construct the dataframe with encodings
        self.dataset = dataset.df
        self.dataset['VAE_ENCODED'] = self.get_encodeings(self.dataset)
        # Load Grid Parameters
        self.hyperparams["tree_params"]["grid_search"] = self.optimize_grid(self.hyperparams["tree_params"]["grid_search"], self.dataset)
        

    def optimize_grid(self, grid_search, df):
        """
        Optimize the grid search parameters
        #@TODO: make it on chunkc of the dataframe and return the top
        #@TODO: the chunkcs in order of encodings
        """
        # Define Decision Tree Classifier
        dec_tree = DecisionTreeClassifier(random_state=0)
        # Define Grid Search
        grid_search = GridSearchCV(dec_tree, grid_search, cv=5, n_jobs=self.hyperparams["tree_params"]["grid_search_jobs"])
        target_values = df[self._mlmodel.data.target]
        train_features = df[self._mlmodel.feature_input_order]
        # Fit the Grid Search
        grid_search.fit(train_features, target_values)
        # Return the best parameters as the format of grid_search
        print(grid_search.best_params_)
        return grid_search.best_params_

    def load_vae(self, data: pd.DataFrame, vae_params: Dict, mlmodel: MLModel, data_name: str) -> VariationalAutoencoder:
        '''
        Load and train the VAE if needed
        '''
        generative_model = VariationalAutoencoder(data_name, vae_params['layers'])
        # if train is True, train the VAE
        if vae_params['train']:
            generative_model = train_variational_autoencoder(
                generative_model,
                data,
                mlmodel.feature_input_order,
                lambda_reg=vae_params["lambda_reg"],
                epochs=vae_params["epochs"],
                lr=vae_params["lr"],
                batch_size=vae_params["batch_size"],
            )
        else:
            try:
                # CHeck if the generative_model can load our data
                generative_model.load(data.shape[1] - 1)
            except FileNotFoundError as exc:
                raise FileNotFoundError(
                    "Loading of Autoencoder failed. {}".format(str(exc))
                )
        
        return generative_model
    
    def get_counterfactuals(self, factuals: pd.DataFrame):
        '''
        this property is responsible to generate and output
        encoded and scaled counterfactual examples
        as pandas DataFrames
        '''
        # Get the encoded features of factuals
        factuals["VAE_ENCODED"] = self.get_encodeings(factuals)
        # Get the counterfactuals
        # find counterfactuals
        counter_factuals = factuals.apply(
            lambda x: self.tree_based_search(x), axis=1, raw=False
        )
        # counter_factuals = [self.tree_based_search(row) for __,row in factuals.iterrows()]
        # Concatenate the counterfactuals to a single dataframe
        # counter_factuals is a list of rows
        self.counter_factuals = counter_factuals
        #counter_factuals = check_counterfactuals(self._mlmodel, counter_factuals)
        # Return the counterfactuals
        return counter_factuals[self._mlmodel.feature_input_order]

    def get_encodeings(self, data: pd.DataFrame):
        '''
        This method is responsible to append the encoded features
        to the dataframe
        '''
        # Fix DataFrame to be able to feed to the VAE
        input_data = data.copy()[self._mlmodel.feature_input_order]
        input_data = torch.FloatTensor(input_data.values)
        # Get the encoded features
        encoded_values = self.vae.encode(input_data)[0].detach().numpy()
        encoded_values = [i for i in encoded_values]
        return encoded_values

    def distance_get(self, x,factuals):
        return np.square((x - factuals)).sum()

    def get_nearest_neighbors_thershold(self, copy_data, label_threshold):
        '''
        This method is responsible to get the nearest neighbors of a given threshold
        using the VAE and minimum threshold per label
        '''
        # Find the index of the 100th instance of each class
        id_100th_class_0 = copy_data[copy_data[self._mlmodel.data.target] == 0].index[label_threshold-1]
        id_100th_class_1 = copy_data[copy_data[self._mlmodel.data.target] == 1].index[label_threshold-1]
        # Get the maximum id
        max_id = max(id_100th_class_0, id_100th_class_1)
        # Return the nearest neighbors of the 100th instance of each class
        return copy_data.head(max_id)
    
    def decision_tree(self, nearest_neighbors):
        '''
        This method is responsible to create a decision tree
        using the nearest neighbors of the 100th instance of each class
        '''
        target_values = nearest_neighbors[self._mlmodel.data.target]
        train_features = nearest_neighbors[self._mlmodel.feature_input_order]
        # Create the decision tree
        clf = DecisionTreeClassifier(random_state=0 , max_depth=self.hyperparams["tree_params"]['grid_search']["max_depth"], 
                                    min_samples_split=self.hyperparams["tree_params"]['grid_search']["min_samples_split"], 
                                    min_samples_leaf=self.hyperparams["tree_params"]['grid_search']["min_samples_leaf"], 
                                    max_features=self.hyperparams["tree_params"]['grid_search']["max_features"])
        # Define the grid search
        #grid_search = GridSearchCV(clf, self.hyperparams["tree_params"]["grid_search"], cv=5, verbose=0, refit=True, n_jobs=self.hyperparams["tree_params"]["grid_search_jobs"])
        # Fit the grid search evaluate on X_test and y_test then refit best model on the whole dataset
        #grid_search.fit(train_features, target_values)
        # Return the best model
        #return grid_search.best_estimator_
        clf.fit(train_features, target_values)
        return clf


    def tree_based_search(self, factual):
        '''
        This method is responsible to get the counterfactual of a given targeted_encoding
        '''
        copy_data = self.dataset.copy()
        # Get distances from data to this encoding
        copy_data["distance"] = copy_data["VAE_ENCODED"].apply(lambda x: self.distance_get(x, factual["VAE_ENCODED"]))
        # Sort the dataframe by distance
        copy_data = copy_data.sort_values(by="distance")
        # Reset the index
        copy_data = copy_data.reset_index(drop=True)
        # Get the nearest neighbors of the targeted encoding
        nearest_neighbors = self.get_nearest_neighbors_thershold(copy_data, label_threshold=self.hyperparams["tree_params"]["min_entries_per_label"])
        # Get the tree
        tree = self.decision_tree(nearest_neighbors)
        self.mtree = tree
        # Get the leaf nodes
        leaf_nodes = TreeLeafs(tree.tree_, self._mlmodel.feature_input_order).leafs_nodes.copy()
        # leaf_nodes is list of classes LeafNode
        # Get the leaf node where the targeted encoding is located
        leaf_node_n_i = -1
        for leaf_i in range(len(leaf_nodes)):
            if leaf_nodes[leaf_i].check_point(factual):
                leaf_node_n_i = leaf_i
                break
        self.mleaf_node_n_i = leaf_node_n_i
        self.mfactual = factual
        # assert if the leaf node is not found
        assert leaf_node_n_i != -1, "Leaf node not found"
        # For now change leafnode label to the item label
        # #assert if leaf_node_n.label is not the same as the factual label
        # assert leaf_node_n.label == factual[self._mlmodel.data.target], "Leaf node label {} is not the same as the factual label {}".format(leaf_node_n.label, factual[self._mlmodel.data.target])
        if leaf_nodes[leaf_node_n_i].label != factual[self._mlmodel.data.target]:
          #print("Leaf Node {} flipped node label {} to match the factual entry {}".format(leaf_node_n_i,
          #                                                                          leaf_nodes[leaf_node_n_i].label,
          #                                                                          factual[self._mlmodel.data.target]))
          leaf_nodes[leaf_node_n_i].label = factual[self._mlmodel.data.target]
        leaf_node_n = leaf_nodes[leaf_node_n_i]
        # Get all leafnodes with label!= leaf_node_n.label and Sort leaf nodes by distance
        leaf_nodes_with_label = [leaf_n for leaf_n in leaf_nodes if leaf_n.label != leaf_node_n.label]
        # Check if leaf_nodes_with_label is empty
        if len(leaf_nodes_with_label) == 0:
            print("No leaf node with label {}".format(leaf_node_n.label))
            factual_ret = factual
            factual_ret[self._mlmodel.feature_input_order] = np.nan
            #print("returned")
            return factual_ret[self._mlmodel.feature_input_order]
        # Sort leaf nodes by distance
        leaf_nodes_with_label = sorted(leaf_nodes_with_label, key=lambda x: len(leaf_node_n.compare_node(x)))
        # Get the counterfactual
        returned_neighbor = -1
        counter_taregt = factual[self._mlmodel.data.target]*-1 +1
        #print("Searching for Neighbor....")
        # print("Start with option A: {}".format(nearest_leaf_node))
        #print(second_nearest_node)
        # If len of leaf_nodes_with_label is 1, the all the max_search on the nearest_leaf_node
        # If len of leaf_nodes_with_label is 2, the max_search/7 on the nearest_leaf_node and max_search/3 on the second_nearest_node
        # If len of leaf_nodes_with_label is 3, the max_search/5 on the nearest_leaf_node and max_search/3 on the second_nearest_node and max_search/2 on the third_nearest_node
        if len(leaf_nodes_with_label) == 1:
            max_searchs = [self.hyperparams["tree_params"]["max_search"]]
        else: #elif len(leaf_nodes_with_label) == 2:
            max_searchs = [self.hyperparams["tree_params"]["max_search"]*0.7, self.hyperparams["tree_params"]["max_search"]*0.3]
        #else:
        #    max_searchs = [self.hyperparams["tree_params"]["max_search"]*0.5, self.hyperparams["tree_params"]["max_search"]*0.3, self.hyperparams["tree_params"]["max_search"]*0.2]
        # map max_search to int values while rounding up to the nearest int
        max_searchs = [int(round(x)) for x in max_searchs]
        # Loop over max_search
        for rank_node, max_search_i in enumerate(max_searchs):
            number_searchs = 0
            nearest_leaf_node = leaf_nodes_with_label[rank_node]
            #print("Searching for Neighbor.... {}, {}".format(rank_node, max_search_i))
            while number_searchs < max_search_i and returned_neighbor is -1:
                # if number_searchs is 30% of max_search
                if number_searchs < max_search_i*0.3:
                    sigma = 20
                    gamma = 0
                # if number_searchs is 60% of max_search
                elif number_searchs < max_search_i*0.6:
                    sigma = 10
                    gamma = 20
                # if number_searchs is 80% of max_search
                elif number_searchs < max_search_i*0.8:
                    sigma = 1
                    gamma = 10
                # if number_searchs is 80% of max_search
                elif number_searchs < max_search_i*0.9:
                    sigma = 0.2
                    gamma = 1
                # if number_searchs is 90% of max_search
                else:
                    sigma = 1
                    gamma = 0

                neighbor = nearest_leaf_node.generate_point(factual.copy(), data_catalog = self.data_catalog, sigma = sigma, gamma = gamma)
                neighb_df = pd.DataFrame([neighbor[self.mlmodel.feature_input_order]])
                self.neighb_df = neighb_df
                probs_p = self.mlmodel.predict_proba(neighb_df)
                if counter_taregt == np.argmax(probs_p):
                    returned_neighbor = neighbor
                    break
                number_searchs += 1
            if returned_neighbor is not -1:
                break
        # If no neighbor is found, return the factual
        if returned_neighbor is -1:
            #print("No neighbor was found")
            factual_ret = factual
            factual_ret[self._mlmodel.feature_input_order] = np.nan
            #print("returned")
            return factual_ret[self._mlmodel.feature_input_order]
        return returned_neighbor[self._mlmodel.feature_input_order]


In [13]:
#TEMP_VAE = tbtest.vae

In [14]:
from carla import MLModelCatalog
from carla.data.catalog import OnlineCatalog
from carla.recourse_methods import GrowingSpheres
from sklearn.model_selection import GridSearchCV, train_test_split

# load a catalog dataset
data_name = "adult"
dataset = OnlineCatalog(data_name)
data_train, data_test = train_test_split(dataset.df, test_size=0.2)

class MyData:
  def __init__(self, data, target):
    self.df = data
    self.target = target
trainData = MyData(data_train.copy(), dataset.target)

# load artificial neural network from catalog
model = MLModelCatalog(dataset, 'ann')
factuals = data_test.head(50)

[WARNING] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor [deprecation.py new_func]
[WARNING] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor [deprecation.py new_func]


In [15]:
# Check our data catalog
col_n = dataset.df.columns
catalog_n = dataset.catalog
# Initialize new catalog
new_catalog_n = {'target': 'income', 'continuous': [], 'categorical': [], 'immutable': []}
# Map continuous values
for col_i in col_n:
    col = col_i.split('_')[0]
    if col == dataset.target:
        continue
    if col in catalog_n['immutable']:
        new_catalog_n['immutable'].append(col_i)
    if col in catalog_n['continuous']:
        new_catalog_n['continuous'].append(col_i)
    elif col in catalog_n['categorical']:
        new_catalog_n['categorical'].append(col_i)
    else:
        assert False, 'Column not found in catalog {}'.format(col_i)

# Assert if new_catalog_n is not same shape as catalog_n
assert len(new_catalog_n['continuous']) == len(catalog_n['continuous']), 'Continuous values not same shape'
assert len(new_catalog_n['categorical']) == len(catalog_n['categorical']), 'Categorical values not same shape'
assert len(new_catalog_n['immutable']) == len(catalog_n['immutable']), 'Immutable values not same shape'
# For each continous value get the std, mean, and min/max and plug them in the new catalog['continuous_stats']
new_catalog_n['continuous_stats'] = {}
for col_i in new_catalog_n['continuous']:
    new_catalog_n['continuous_stats'][col_i] = {}
    new_catalog_n['continuous_stats'][col_i]['std'] = data_train[col_i].std()
    new_catalog_n['continuous_stats'][col_i]['mean'] = data_train[col_i].mean()
    new_catalog_n['continuous_stats'][col_i]['min'] = data_train[col_i].min()
    new_catalog_n['continuous_stats'][col_i]['max'] = data_train[col_i].max()


In [16]:
from carla.models.catalog import MLModelCatalog

training_params = {"lr": 0.001, "epochs": 3, "batch_size": 64, "hidden_size": [10, 5]}

ml_model = MLModelCatalog(
    dataset,
    model_type="ann",
    load_online=False,
    backend="tensorflow"
)

ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"]
)

balance on test set 0.23883245958934032, balance on test set 0.2408256880733945
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                140       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 12        
Total params: 207
Trainable params: 207
Non-trainable params: 0
_________________________________________________________________
None
[WARNING] From /content/CARLA/carla/models/catalog/ANN_TF/model_ann.py:10: The name tf.log is deprecated. Please use tf.math.log instead.
 [deprecation_wrapper.py __getattr__]
[WARNING] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables i

In [33]:
hpr = {
      "data_name": "data_name",
      "n_search_samples": 300,
      "p_norm": 1,
      "step": 0.1,
      "max_iter": 10,
      "clamp": True,
      "binary_cat_features": True,
      "vae_params": {
          "layers": [len(model.feature_input_order), 20, 10, 7],
          "train": True,
          "lambda_reg": 1e-6,
          "epochs": 5,
          "lr": 1e-3,
          "batch_size": 16,
      },
      "tree_params": {
          "min_entries_per_label": 1000,
          "grid_search_jobs": -1,
          "min_weight_gini": 100, # set to 0.5 since here both class have same prob,
          "max_search" : 500,
          "grid_search": {
                "cv": 1,
                "splitter": ["best","random"],
                "criterion": ["gini"],
                "max_depth": [2,4,6,8,10,12,14],
                "min_samples_split": [1,2,3,4],
                "min_samples_leaf": [1,2,3,4],
                "max_features": [None] #Note changing this will result in removing features that we might want to keep
          }
      }
    }

# Conditions Violations add it at the beginning before going to sampling
# Immutable, e.g. gender imutable all directions
# Age should be greater than x okay, bs decrease 
# Our method take care of those imutability
# 
# Decision tree validation train 
# DOn't cross. validation
# Don't retrain
# Just to retrain
#julia here
tbtest = TreeBasedContrastiveExplanation(trainData, model, hpr, data_catalog= new_catalog_n)
#skk = tbtest.get_counterfactuals(factuals.copy().head(10))
benchmark = Benchmark(model, tbtest, factuals.copy().head(5).copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

[INFO] Start training of Variational Autoencoder... [models.py fit]
[INFO] [Epoch: 0/5] [objective: 0.524] [models.py fit]
[INFO] [ELBO train: 0.52] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] [ELBO train: 0.11] [models.py fit]
[INFO] [ELBO train: 0.11] [models.py fit]
[INFO] [ELBO train: 0.11] [models.py fit]
[INFO] ... finished training of Variational Autoencoder. [models.py fit]
{'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'splitter': 'best'}


,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,2.0,0.575259,0.232124,0.470198,1,2,0.3,0.8,0.944848


In [34]:
check_data = data_test.sample(100)

In [37]:
benchmark = Benchmark(model, tbtest, check_data.copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,2.0,0.15711,0.014,0.107349,0,2,0.333333,0.54,1.486302


In [35]:
gs = GrowingSpheres(model)
benchmark = Benchmark(model, gs,  check_data.copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,6.0,1.097119,1.003558,1.0,0,3,0.156757,0.74,0.055225


In [36]:
hyperparams = setup["dice"]["hyperparams"]
dice = Dice(model, hyperparams)
benchmark = Benchmark(model, dice,  check_data.copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,2.0,1.257236,0.937618,0.9,1,1,0.704,1.0,0.135743


In [40]:
from carla.recourse_methods import CCHVAE
hyperparams = setup["cchvae"]["hyperparams"]
hyperparams["data_name"] = data_name
hyperparams["vae_params"]["layers"] = [
    len(ml_model.feature_input_order)
] + hyperparams["vae_params"]["layers"]
cchvae = CCHVAE(ml_model, hyperparams)

[INFO] Start training of Variational Autoencoder... [models.py fit]
[INFO] [Epoch: 0/5] [objective: 0.388] [models.py fit]
[INFO] [ELBO train: 0.39] [models.py fit]
[INFO] [ELBO train: 0.13] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] ... finished training of Variational Autoencoder. [models.py fit]


In [41]:
benchmark = Benchmark(ml_model, cchvae,  check_data.copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,8.0,2.310385,2.027238,1.0,1,3,0.0,0.82,1.512377


In [42]:
from carla.recourse_methods import CRUD
hyperparams = setup["cruds"]["hyperparams"]
hyperparams["data_name"] = data_name
hyperparams["vae_params"]["layers"] = [
    len(ml_model.feature_input_order)
] + hyperparams["vae_params"]["layers"]
cruds = CRUD(ml_model, hyperparams)
benchmark = Benchmark(ml_model, cruds,  check_data.copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

[INFO] Start training of CSVAE... [models.py fit]


  0%|          | 0/5 [00:00<?, ?it/s]

[INFO] epoch 0: x recon loss: 0.15420975030777026 [models.py fit]
[INFO] epoch 0: y recon loss: 29.939648405752518 [models.py fit]


 20%|██        | 1/5 [00:26<01:45, 26.36s/it]

[INFO] epoch 1: x recon loss: 0.13359600185205475 [models.py fit]
[INFO] epoch 1: y recon loss: 26.66030174407859 [models.py fit]


...

 80%|████████  | 4/5 [01:53<00:29, 29.38s/it]

[INFO] epoch 4: x recon loss: 0.11627786827040532 [models.py fit]
[INFO] epoch 4: y recon loss: 24.671035566617434 [models.py fit]


100%|██████████| 5/5 [02:20<00:00, 28.08s/it]

[INFO] ... finished training of CSVAE [models.py fit]


,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,10.0,4.486466,4.0697,1.0,1,9,1.0,1.0,6.699712


In [ ]:
from carla.recourse_methods import Revise
hyperparams = setup["revise"]["hyperparams"]
hyperparams["data_name"] = data_name
hyperparams["vae_params"]["layers"] = [
    len(ml_model.feature_input_order)
] + hyperparams["vae_params"]["layers"]
revise = Revise(ml_model, trainData, hyperparams)
benchmark = Benchmark(ml_model, revise,  check_data.copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

[INFO] Start training of Variational Autoencoder... [models.py fit]
[INFO] [Epoch: 0/5] [objective: 0.344] [models.py fit]
[INFO] [ELBO train: 0.34] [models.py fit]
[INFO] [ELBO train: 0.14] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] ... finished training of Variational Autoencoder. [models.py fit]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] No counterfactual found [model.py _counterfactual_optimization]
[INFO] No counterfactual f

In [48]:
benchmark.run_benchmark().head(1)

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,4.0,1.540268e-08,1.483503e-16,1.192093e-08,0,4,0.888889,0.18,12.764061


In [52]:
from carla.recourse_methods import Wachter
hyperparams = setup["wachter"]["hyperparams"]
wachter = Wachter(ml_model, hyperparams)
benchmark = Benchmark(ml_model, wachter,  check_data.copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactua

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,6.0,1.046093,0.182387,0.175646,1,0,0.212,1.0,0.033903


In [23]:
tbtest2 = TreeBasedContrastiveExplanation(trainData, ml_model, hpr, data_catalog= new_catalog_n)
skk = tbtest2.get_counterfactuals(factuals.copy().head(10))

[INFO] Start training of Variational Autoencoder... [models.py fit]
[INFO] [Epoch: 0/5] [objective: 0.553] [models.py fit]
[INFO] [ELBO train: 0.55] [models.py fit]
[INFO] [ELBO train: 0.12] [models.py fit]
[INFO] [ELBO train: 0.11] [models.py fit]
[INFO] [ELBO train: 0.11] [models.py fit]
[INFO] [ELBO train: 0.11] [models.py fit]
[INFO] ... finished training of Variational Autoencoder. [models.py fit]
{'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'splitter': 'best'}


In [26]:
benchmark = Benchmark(ml_model, tbtest, data_test.sample(10).copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,4.0,1.437507,1.11789,1.0,1,1,0.6,0.7,1.236451


In [38]:
from carla.models.catalog import MLModelCatalog

training_params = {"lr": 0.001, "epochs": 3, "batch_size": 64, "hidden_size": [10, 5]}

ml_model = MLModelCatalog(
    dataset,
    model_type="ann",
    load_online=False,
    backend="pytorch"
)

ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"]
)

Loaded model from /root/carla/models/adult/ann_layers_10_5.pt
test accuracy for model: 0.8392038007863696


In [ ]:
hyperparams["data_name"] = data_name
hyperparams["vae_params"]["layers"] = [
    len(mlmodel.feature_input_order)
] + hyperparams["vae_params"]["layers"]
cchvae = CCHVAE(ml_model, hyperparams)

In [28]:
#### NEW 2 BENCHMARKING
benchmark = Benchmark(model, tbtest, check_data.copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,4.0,1.313009,1.036462,1.0,0,3,0.364706,0.68,1.153777


In [29]:
#### NEW 2 BENCHMARKING
benchmark = Benchmark(model, gs, check_data.copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,7.0,2.249006,2.022722,1.0,0,5,0.119512,0.82,0.021501


In [30]:
hyperparams = setup["dice"]["hyperparams"]
dice = Dice(model, hyperparams)
benchmark = Benchmark(model, dice, check_data.copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head(1)

,Distance_1,Distance_2,Distance_3,Distance_4,Constraint_Violation,Redundancy,y-Nearest-Neighbours,Success_Rate,Average_Time
0,2.0,1.518163,1.155758,0.8,1,1,0.638,1.0,0.133106


In [31]:
hyperparams = setup["face_knn"]["hyperparams"]
face = Face(model, hyperparams)
benchmark = Benchmark(model, face, data_test.head(500).copy().reset_index(drop=True))
distances = benchmark.compute_distances()
benchmark.run_benchmark().head()

KeyboardInterrupt: ignored

In [ ]:
from carla import log
class ARGSS:
  def __init__(self):
    self.recourse_method = ["cchvae", "clue", "cruds", "wachter", "revise"]
    self.dataset = ['adult']
    self.type = ['ann']
    self.number_of_samples = 10

args = ARGSS()


results = pd.DataFrame()
session_models = ["cem", "cem-vae"]
torch_methods = [ "cruds", "wachter", "revise"]
for rm in args.recourse_method:
    backend = "tensorflow"
    if rm in torch_methods:
        backend = "pytorch"
    for data_name in args.dataset:
        dataset = OnlineCatalog(data_name)
        for model_type in args.type:
            log.info("=====================================")
            log.info("Recourse method: {}".format(rm))
            log.info("Dataset: {}".format(data_name))
            log.info("Model type: {}".format(model_type))

            if rm in session_models:
                graph = Graph()
                with graph.as_default():
                    ann_sess = Session()
                    with ann_sess.as_default():
                        mlmodel_sess = MLModelCatalog(dataset, model_type, backend)

                        factuals_sess = predict_negative_instances(
                            mlmodel_sess, dataset
                        )
                        factuals_sess = factuals_sess.iloc[: args.number_of_samples]
                        factuals_sess = factuals_sess.reset_index(drop=True)

                        recourse_method_sess = initialize_recourse_method(
                            rm,
                            mlmodel_sess,
                            dataset,
                            data_name,
                            model_type,
                            setup,
                            sess=ann_sess,
                        )

                        df_benchmark = Benchmark(
                            mlmodel_sess, recourse_method_sess, factuals_sess
                        ).run_benchmark()
            else:
                mlmodel = MLModelCatalog(dataset, model_type, backend)

                factuals = predict_negative_instances(mlmodel, dataset.df)
                factuals = factuals.iloc[: args.number_of_samples]
                factuals = factuals.reset_index(drop=True)

                recourse_method = initialize_recourse_method(
                    rm, mlmodel, dataset, data_name, model_type, setup
                )

                df_benchmark = Benchmark(
                    mlmodel, recourse_method, factuals
                ).run_benchmark()

            df_benchmark["Recourse_Method"] = rm
            df_benchmark["Dataset"] = data_name
            df_benchmark["ML_Model"] = model_type
            df_benchmark = df_benchmark[
                [
                    "Recourse_Method",
                    "Dataset",
                    "ML_Model",
                    "Distance_1",
                    "Distance_2",
                    "Distance_3",
                    "Distance_4",
                    "Constraint_Violation",
                    "Redundancy",
                    "y-Nearest-Neighbours",
                    "Success_Rate",
                    "Average_Time",
                ]
            ]

            results = pd.concat([results, df_benchmark], axis=0)
            log.info("=====================================")
